# jaccard gwas

In [2]:
import numpy as np
import scipy.sparse as sp
# import h5py
import re
import pandas as pd

def jaccard_ind(set1, set2):
    intersect = list(set(set1).intersection(set(set2)))
    union = list(set(set1).union(set(set2)))
    try:
        ret = len(intersect) / len(union)
    except:
        ret = 0
    return ret

fr = open(r"./disease gene reprioritization/gwas_cui_MAPPED_TRAIT_threshold_30_tab.txt", "r")
ll = fr.readlines()

df_mental_list = []

mental_dict = {"C0041696":"unipolar depression",
               "C0038436":"post-traumatic stress disorder",
               "C1263846":"attention deficit hyperactivity disorder",
               "C0005586":"bipolar disorder",
               "C0036341":"schizophrenia",
               "C1510586":"autism spectrum disorder",
               "C0040517":"Tourette syndrome",
               "C0003125":"anorexia nervosa"
               }
mental_dict_gene = {}
for key in list(mental_dict.keys()):
    for l in ll:
        line = l.strip().split("\t")
        if line[0] == key:
            mental_dict_gene[key] = line[2:]

for id,name in mental_dict.items():
    for id2,name2 in mental_dict.items():
        if id2 != id:
            value = jaccard_ind(mental_dict_gene[id],mental_dict_gene[id2])
            df_mental_list.append({"dis1":name,"dis2":name2,"value":value})

df2 = pd.DataFrame(df_mental_list)
df2.to_csv(r"./mental_8_R.csv", index = False)


# jaccard disgeNet

In [3]:
import numpy as np
import scipy.sparse as sp
# import h5py
import re
import pandas as pd

def jaccard_ind(set1, set2):
    intersect = list(set(set1).intersection(set(set2)))
    union = list(set(set1).union(set(set2)))
    try:
        ret = len(intersect) / len(union)
    except:
        ret = 0
    return ret

fr1 = open(r"./disease gene reprioritization/disgenet.curated.v7.entrez.gmt", "r")
ll1 = fr1.readlines()
fr2 = open(r"./disease gene reprioritization/disgenet.all.v7.entrez.gmt", "r")
ll2 = fr2.readlines()

df_mental_list = []

mental_dict = {"C0041696":"unipolar depression",
               "C0038436":"post-traumatic stress disorder",
               "C1263846":"attention deficit hyperactivity disorder",
               "C0005586":"bipolar disorder",
               "C0036341":"schizophrenia",
               "C1510586":"autism spectrum disorder",
               "C0040517":"Tourette syndrome",
               "C0003125":"anorexia nervosa"
               }
mental_dict_gene = {}
sign1 = 0
for key in list(mental_dict.keys()):
    sign1 = 0
    for l in ll1:
        line = l.strip().split("\t")
        if line[0] == key:
            sign1 = 1
            mental_dict_gene[key] = line[2:]
    if sign1 == 0:
        for l in ll2:
            line = l.strip().split("\t")
            if line[0] == key:
                sign1 = 1
                mental_dict_gene[key] = line[2:]
    if sign1 == 0:
        print("error")

for id,name in mental_dict.items():
    for id2,name2 in mental_dict.items():
        if id2 != id:
            value = jaccard_ind(mental_dict_gene[id],mental_dict_gene[id2])
            df_mental_list.append({"dis1":name,"dis2":name2,"value":value})
print("haha")
df2 = pd.DataFrame(df_mental_list)
df2.to_csv(r"./mental_8_disgeNet_R.csv", index = False)


# jaccard gat norm

In [4]:
import numpy as np
import scipy.sparse as sp
# import h5py
import re
import pandas as pd
import json


# the number of TPGs was chosen according to their GWAS association genes in training, 
# and normalized to a range from 100 to 500
def lengh_normalize(figu,low,up):
    lower = low
    upper = up
    k = (upper-lower)/(914-33)
    ret = int(lower + k*(figu - 33))
    return ret

#map json
fj =open(r"./network_data/map.json",'r')
json_map = json.load(fj)
print(json_map[0])
map_dict = json_map[0]

def jaccard_ind(set1, set2):
    intersect = list(set(set1).intersection(set(set2)))
    union = list(set(set1).union(set(set2)))
    try:
        ret = len(intersect) / len(union)
    except:
        ret = 0
    return ret

arr = np.load(r"./disease gene reprioritization/result/logits202.npy")
fr = open(r"./disease gene reprioritization/gwas_cui_MAPPED_TRAIT_threshold_30_tab.txt")
ll = fr.readlines()

df_mental_list = []

mental_dict = {"C0041696":"unipolar depression",
               "C0038436":"post-traumatic stress disorder",
               "C1263846":"attention deficit hyperactivity disorder",
               "C0005586":"bipolar disorder",
               "C0036341":"schizophrenia",
               "C1510586":"autism spectrum disorder",
               "C0040517":"Tourette syndrome",
               "C0003125":"anorexia nervosa"
               }
mental_dis_ind = {}
mental_dis_gene_num = {}
for key,value in mental_dict.items():
    for i in range(len(ll)):
        line = ll[i].strip().split("\t")
        if line[0] == key:
            mental_dis_ind[value] = i
            mental_dis_gene_num[value] = lengh_normalize(len(line[2:]),100,500)  #normalized to a range from 100 to 500
mental_dis_gene = {}


for name,ind in mental_dis_ind.items():
    dis = arr[ind].tolist()
    dis_dict = dict(zip(range(18547), dis))
    or_dis_dict = sorted(dis_dict.items(), key=lambda d: d[1], reverse=True)
    selected_L = or_dis_dict[:mental_dis_gene_num[name]]  # select top genes
    #selected_L = or_dis_dict[:100]  # select top genes
    tmp_genes = []
    for item in selected_L:
        gene = map_dict[str(item[0])]
        tmp_genes.append(gene)
    mental_dis_gene[name] = tmp_genes

for name,genes in mental_dis_gene.items():
    for name2,genes2 in mental_dis_gene.items():
        if name != name2:
            value = jaccard_ind(genes,genes2)
            df_mental_list.append({"dis1":name,"dis2":name2,"value":value})

df2 = pd.DataFrame(df_mental_list)
df2.to_csv(r"./mental_8_GatLogit_normLong_R1.csv", index = False)




